### Package

In [ ]:
# Package
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time

### Scrapping

In [ ]:
# List Penyimpanan
daftarHarga = []
daftarLokasi = []
daftarLT = []
daftarLB = []
daftarKamar = []
daftarKamarMandi = []
daftarCarport = []

# buat instance webdriver
driver = webdriver.Chrome()

for i in range(1, 11):
    url = f'https://www.rumah123.com/jual/bekasi/rumah/?page={i}'
    driver.get(url)

    # Scroll untuk load 
    for _ in range(24):

        # Scroll awah
        driver.execute_script("window.scrollBy(0, 1000)")
        time.sleep(1)

    # inisialisasi html
    html = driver.page_source

    # inisialisasi BeutifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # inisiasi bagian discraping
    homes= soup.find_all('div',class_='card-featured__middle-section')

    for home in homes:

        # bagian harga
        hargaRumah = home.find('div' , class_="card-featured__middle-section__price")
        harga = hargaRumah.find('strong').get_text()
        daftarHarga.append(harga)

        # bagian lokasi
        lokasiRumah = home.find("a", title=True, href=True)
        lokasi = lokasiRumah.find_next_sibling("span").get_text()
        daftarLokasi.append(lokasi)

        # inisiasi bagian LT dan LB
        atribute_info = home.find_all("div", {"class": "attribute-info"})  # harus find_all
        # cek panjang dulu biar aman
        if len(atribute_info) > 0:
            lt = atribute_info[0].find("span").get_text()
            daftarLT.append(lt)
        if len(atribute_info) > 1:
            lb = atribute_info[1].find("span").get_text()
            daftarLB.append(lb)
        
        # inisiasi bagian Kamar , Kamar mandi dan Carport
        atribute = home.find_all("div", {"class": "relative ui-molecules-list__item"})
        # kamar tidur
        if len(atribute) > 0:
            kamar = atribute[0].find('span', class_='attribute-text').get_text()
            daftarKamar.append(kamar)
        # kamar mandi
        if len(atribute) > 1:
            kamarMandi = atribute[1].find('span', class_='attribute-text').get_text()
            daftarKamarMandi.append(kamarMandi)
        # carport
        if len(atribute) > 2:
            carport = atribute[2].find('span', class_='attribute-text').get_text()
            daftarCarport.append(carport)
        else:
            carport = None
            daftarCarport.append(carport)
driver.close()

### Save

In [ ]:
# Convert Dataframe
data= ({
    "Harga": daftarHarga,
    "Lokasi": daftarLokasi,
    "LT": daftarLT,
    "LB": daftarLB,
    "Kamar": daftarKamar,
    "Kamar Mandi": daftarKamarMandi,
    "Carport": daftarCarport
})

# Save Csv
df = pd.DataFrame(data).to_csv("rumah_bekasi.csv", index=False)
df.head
